# Import

In [29]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.optimize import minimize
from scipy.interpolate import interp1d

# RSM ciągły

In [30]:
def continuous_reference_set_method(alternatives, criteria_types, a, b, n_extra=200):
    rng = np.random.default_rng()
    M = alternatives.shape[1]
    extra_points = np.zeros((n_extra, M))
    for i in range(M):
        low = min(a[i], b[i])
        high = max(a[i], b[i])
        extra_points[:, i] = rng.uniform(low, high, n_extra)
    all_points = np.vstack([alternatives, extra_points])
    def normalize(points, a, b, criteria_types):
        norm_alt = np.zeros_like(points, dtype=float)
        for i in range(len(criteria_types)):
            if criteria_types[i] == -1:  # Minimalizacja
                norm_alt[:, i] = (points[:, i] - a[i]) / (b[i] - a[i])
            else:  # Maksymalizacja
                norm_alt[:, i] = (b[i] - points[:, i]) / (b[i] - a[i])
        return norm_alt
    def scoring_function(points, a, b, criteria_types):
        norm_alt = normalize(points, a, b, criteria_types)
        distance_to_a = np.linalg.norm(norm_alt - np.zeros((len(points), len(a))), axis=1)
        distance_to_b = np.linalg.norm(norm_alt - np.ones((len(points), len(a))), axis=1)
        C = distance_to_b / (distance_to_a + distance_to_b)
        return C
    scores = scoring_function(all_points, a, b, criteria_types)
    ranking = np.argsort(-scores)  # sortujemy malejąco po scores
    return ranking, scores, all_points


alternatives = np.array([
    [5, 7, 6],
    [3, 8, 5],
    [6, 6, 7],
    [2, 9, 4],
    [4, 5, 8],
    [7, 4, 5],
    [5, 6, 6],
    [6, 5, 7]
])

directions = np.array([-1, 1, -1])
a = np.array([2, 9, 4])
b = np.array([7, 4, 8])

ranking, scores, all_points = continuous_reference_set_method(alternatives, directions, a, b, n_extra=10)
print("\nTest case 3 - Results:")
print("|  Alternative   | Score   | Rank  |")
print("|----------------|---------|-------|")
for i, alternative in enumerate(ranking):
    print(f"| Alternative {alternative + 1:<2} | {scores[alternative]:<7.4f} | {i + 1:<5} |")


Test case 3 - Results:
|  Alternative   | Score   | Rank  |
|----------------|---------|-------|
| Alternative 12 | 0.6738  | 1     |
| Alternative 18 | 0.5977  | 2     |
| Alternative 9  | 0.5896  | 3     |
| Alternative 4  | 0.5858  | 4     |
| Alternative 2  | 0.5640  | 5     |
| Alternative 6  | 0.5481  | 6     |
| Alternative 17 | 0.5465  | 7     |
| Alternative 10 | 0.5172  | 8     |
| Alternative 7  | 0.5000  | 9     |
| Alternative 15 | 0.4940  | 10    |
| Alternative 16 | 0.4923  | 11    |
| Alternative 5  | 0.4772  | 12    |
| Alternative 14 | 0.4625  | 13    |
| Alternative 11 | 0.4368  | 14    |
| Alternative 8  | 0.4360  | 15    |
| Alternative 1  | 0.4339  | 16    |
| Alternative 3  | 0.3681  | 17    |
| Alternative 13 | 0.2960  | 18    |


# RSM dyskretny

In [31]:
def discrete_reference_set_method(alternatives, criteria_types, a, b):
    def normalize(alternatives, a, b, criteria_types):
        norm_alt = np.zeros_like(alternatives, dtype=float)
        for i in range(len(criteria_types)):
            if criteria_types[i] == -1:
                norm_alt[:, i] = (alternatives[:, i] - a[i]) / (b[i] - a[i])
            else:
                norm_alt[:, i] = (b[i] - alternatives[:, i]) / (b[i] - a[i])
        return norm_alt
    def scoring_function(alternatives, a, b, criteria_types):
        norm_alt = normalize(alternatives, a, b, criteria_types)
        distance_to_a = np.linalg.norm(norm_alt - np.zeros((len(alternatives), len(a))), axis=1)
        distance_to_b = np.linalg.norm(norm_alt - np.ones((len(alternatives), len(a))), axis=1)
        C = distance_to_b / (distance_to_a + distance_to_b)
        return C
    scores = scoring_function(alternatives, a, b, criteria_types)
    ranking = np.argsort(-scores)
    return ranking, scores

alternatives = np.array([
    [5, 7, 6],
    [3, 8, 5],
    [6, 6, 7],
    [2, 9, 4],
    [4, 5, 8],
    [7, 4, 5],
    [5, 6, 6],
    [6, 5, 7]
])

directions = np.array([-1, 1, -1])
a = np.array([2, 9, 4])
b = np.array([7, 4, 8])

ranking, scores = discrete_reference_set_method(alternatives, directions, a, b)
print("\nTest case 3 - Results:")
print("|  Alternative   | Score   | Rank  |")
print("|----------------|---------|-------|")
for i, alternative in enumerate(ranking):
    print(f"| Alternative {alternative + 1:<2} | {scores[alternative]:<7.4f} | {i + 1:<5} |")


Test case 3 - Results:
|  Alternative   | Score   | Rank  |
|----------------|---------|-------|
| Alternative 4  | 0.5858  | 1     |
| Alternative 2  | 0.5640  | 2     |
| Alternative 6  | 0.5481  | 3     |
| Alternative 7  | 0.5000  | 4     |
| Alternative 5  | 0.4772  | 5     |
| Alternative 8  | 0.4360  | 6     |
| Alternative 1  | 0.4339  | 7     |
| Alternative 3  | 0.3681  | 8     |


# Fuzzy Topsis

In [24]:
def fuzzy_topsis(decision_matrix, weights, criteria_types):
    norm_matrix = np.zeros_like(decision_matrix, dtype=float)
    for j in range(decision_matrix.shape[1]):
        if criteria_types[j] == 1:
            max_value = np.max(decision_matrix[:, j, 2])
            norm_matrix[:, j, 0] = decision_matrix[:, j, 0] / max_value
            norm_matrix[:, j, 1] = decision_matrix[:, j, 1] / max_value
            norm_matrix[:, j, 2] = decision_matrix[:, j, 2] / max_value
        else:
            min_value = np.min(decision_matrix[:, j, 0])
            norm_matrix[:, j, 0] = min_value / decision_matrix[:, j, 2]
            norm_matrix[:, j, 1] = min_value / decision_matrix[:, j, 1]
            norm_matrix[:, j, 2] = min_value / decision_matrix[:, j, 0]
    weighted_matrix = np.zeros_like(norm_matrix, dtype=float)
    for j in range(norm_matrix.shape[1]):
        weighted_matrix[:, j, 0] = norm_matrix[:, j, 0] * weights[j]
        weighted_matrix[:, j, 1] = norm_matrix[:, j, 1] * weights[j]
        weighted_matrix[:, j, 2] = norm_matrix[:, j, 2] * weights[j]
    fpis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    fnis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    for j in range(weighted_matrix.shape[1]):
        if criteria_types[j] == 1:
            fpis[j] = np.max(weighted_matrix[:, j, :], axis=0)
            fnis[j] = np.min(weighted_matrix[:, j, :], axis=0)
        else:
            fpis[j] = np.min(weighted_matrix[:, j, :], axis=0)
            fnis[j] = np.max(weighted_matrix[:, j, :], axis=0)
    distance_to_fpis = np.sqrt(np.sum(np.power(weighted_matrix - fpis, 2), axis=(1, 2)))
    distance_to_fnis = np.sqrt(np.sum(np.power(weighted_matrix - fnis, 2), axis=(1, 2)))
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity_to_ideal = distance_to_fnis / (distance_to_fpis + distance_to_fnis)
        similarity_to_ideal[np.isnan(similarity_to_ideal)] = 0
    ranking = np.argsort(similarity_to_ideal)[::-1]
    return similarity_to_ideal, ranking


decision_matrix_fuzzy = np.array([
            [[5, 6, 7], [3, 4, 5], [1, 2, 3], [2, 3, 4]],
            [[4, 5, 6], [4, 5, 6], [2, 3, 4], [3, 4, 5]],
            [[3, 4, 5], [5, 6, 7], [3, 4, 5], [4, 5, 6]],
            [[2, 3, 4], [6, 7, 8], [4, 5, 6], [5, 6, 7]],
            [[1, 2, 3], [7, 8, 9], [5, 6, 7], [6, 7, 8]]
        ])

weights = [0.2, 0.4, 0.3, 0.1]
# criteria_type = ['max', 'max', 'min', 'max']
criteria_type = [1, 1, -1, 1]

score, ranking = fuzzy_topsis(decision_matrix_fuzzy, weights, criteria_type)

print("\nTest case 3 - Results:")
print("| Alternative | Score   | Rank  |")
print("|-------------|---------|-------|")
for i, alternative in enumerate(ranking):
    print(f"| Alternative {alternative + 1:<2} | {score[alternative]:<7.4f} | {i + 1:<5} |")


Test case 3 - Results:
| Alternative | Score   | Rank  |
|-------------|---------|-------|
| Alternative 3  | 0.5962  | 1     |
| Alternative 4  | 0.6723  | 2     |
| Alternative 5  | 0.6777  | 3     |
| Alternative 1  | 0.3223  | 4     |
| Alternative 2  | 0.4649  | 5     |


# Topsis

In [25]:
def topsis(decision_matrix, weights, criteria):
    """
    Implementation of the TOPSIS method for multicriteria decision analysis in discrete form.

    Parameters:
    decision_matrix (ndarray): Matrix (m x n) with m alternatives and n criteria.
    weights (list or ndarray): Weights for each criterion.
    criteria (list or ndarray): A list specifying whether each criterion should be maximized (1) or minimized (-1).

    Returns:
    tuple: Ranking of alternatives and their respective scores.
    """
    # Step 1: Normalize the decision matrix
    norm_matrix = decision_matrix / np.sqrt((decision_matrix ** 2).sum(axis=0))

    # Step 2: Apply weights to the normalized matrix
    weighted_matrix = norm_matrix * weights

    # Step 3: Determine the ideal (best) and anti-ideal (worst) solutions
    ideal_best = np.max(weighted_matrix, axis=0) * (criteria == 1) + np.min(weighted_matrix, axis=0) * (criteria == -1)
    ideal_worst = np.min(weighted_matrix, axis=0) * (criteria == 1) + np.max(weighted_matrix, axis=0) * (criteria == -1)

    # Step 4: Calculate distances from the ideal and anti-ideal solutions
    dist_to_ideal = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
    dist_to_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))

    # Step 5: Calculate the score for each alternative
    score = dist_to_worst / (dist_to_ideal + dist_to_worst)

    # Step 6: Rank the alternatives
    ranking = np.argsort(score)[::-1]  # Sort in descending order by score
    return ranking, score

decision_matrix = np.array([
    [250, 16, 12, 5],
    [200, 16, 8, 3],
    [300, 32, 16, 4],
    [275, 32, 8, 4],
    [225, 16, 16, 2]
])

weights = [0.2, 0.4, 0.3, 0.1]
criteria = np.array([1, 1, 1, -1])

ranking, score = topsis(decision_matrix, weights, criteria)

print("\nTest case 3 - Results:")
print("| Alternative | Score   | Rank  |")
print("|-------------|---------|-------|")
for i, alternative in enumerate(ranking):
    print(f"| Alternative {alternative + 1:<2} | {score[alternative]:<7.4f} | {i + 1:<5} |")


Test case 3 - Results:
| Alternative | Score   | Rank  |
|-------------|---------|-------|
| Alternative 3  | 0.8646  | 1     |
| Alternative 4  | 0.5811  | 2     |
| Alternative 5  | 0.4304  | 3     |
| Alternative 1  | 0.2570  | 4     |
| Alternative 2  | 0.1354  | 5     |


# UTAstar dyskretna

In [27]:
def UTA_function_discrete(values, reference_points, lambda_param=1):
    """
    Aproksymuje funkcje użyteczności dla wersji dyskretnej.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_values = []
    
    # Iterujemy przez alternatywy
    for val in values:
        alternative_utility = []
        
        # Iterujemy przez każde kryterium
        for i in range(len(val)):
            ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
            # Interpolacja liniowa dla każdego kryterium
            interp_func = interp1d(ref_values, [-1, 0, 1], kind='linear', fill_value="extrapolate")
            utility_value = interp_func(val[i]) * lambda_param  # Użycie λ
            alternative_utility.append(utility_value)
        
        utility_values.append(alternative_utility)
    
    return np.array(utility_values)

# Funkcja do porównania alternatyw (dyskretna wersja)
def UTAstar_discrete(values, reference_points, lambda_param=1):
    """
    Metoda UTA Star dla wersji dyskretnej (wielokryterialnej) uwzględniająca preferencje decydenta.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_discrete(values, reference_points, lambda_param)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution_idx = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution_idx, utility_values


# Przykład użycia (wersja ciągła)
values_discrete = np.array([
    [10, 20, 30, 60],
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points = np.array([
    [80, 90, 100, 10],  # Najgorszy
    [50, 60, 70, 80],   # Neutralny
    [20, 30, 40, 55],  # Najlepszy
])

lambda_param = 1

best_solution_discrete_idx, utility_values_discrete = UTAstar_discrete(values_discrete, reference_points, lambda_param=lambda_param)
print(f"Najlepsza alternatywa (dyskretna): {values_discrete[best_solution_discrete_idx]} z funkcją użyteczności {utility_values_discrete[best_solution_discrete_idx]}")

Najlepsza alternatywa (dyskretna): [10 20 30 60] z funkcją użyteczności [1.33333333 1.33333333 1.33333333 0.8       ]


# UTAstar continous

In [28]:
def UTA_function_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Aproksymuje funkcje użyteczności dla wersji ciągłej.

    :param values: Lista wartości alternatyw w przestrzeni kryteriów (ciągłe wartości)
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Flaga, czy używać interpolacji wielomianowej zamiast liniowej
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_functions = []

    # Budowa funkcji użyteczności dla każdego kryterium
    for i in range(len(reference_points[0])):
        ref_values = [ref[i] for ref in reference_points]  # Punkty referencyjne dla kryterium i
        utility_values = [-1, 0, 1]  # Wartości użyteczności odpowiadające punktom referencyjnym

        if use_polynomial:
            # Interpolacja wielomianowa za pomocą np.polyfit
            coeffs = np.polyfit(ref_values, utility_values, deg=min(len(ref_values)-1, 2))
            def poly_func(x, coeffs=coeffs):
                return np.polyval(coeffs, x)
            utility_functions.append(poly_func)
        else:
            # Tworzenie funkcji aproksymującej (ciągła interpolacja liniowa)
            interp_func = interp1d(ref_values, utility_values, kind='linear', fill_value="extrapolate")
            utility_functions.append(interp_func)

    # Wyznaczanie wartości funkcji użyteczności dla alternatyw
    utility_values = []
    for val in values:
        alternative_utility = []
        for i in range(len(val)):
            utility_value = utility_functions[i](val[i]) * lambda_param  # Obliczanie użyteczności dla kryterium i
            alternative_utility.append(utility_value)
        utility_values.append(alternative_utility)

    return np.array(utility_values)

# Funkcja optymalizacji w wersji ciągłej
def UTAstar_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Metoda UTA Star dla wersji ciągłej (wielokryterialnej).

    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Flaga, czy używać interpolacji wielomianowej zamiast liniowej
    :return: Najlepsza alternatywa oraz jej użyteczność
    """
    # Wyznaczanie bounds na podstawie wartości alternatyw
    bounds = [(np.min(values[:, i]), np.max(values[:, i])) for i in range(values.shape[1])]

    # Funkcja celu: Maksymalizacja sumy użyteczności
    def objective_function(x):
        utility_values = UTA_function_continuous([x], reference_points, lambda_param=lambda_param, use_polynomial=use_polynomial)
        return -np.sum(utility_values)  # Negujemy, ponieważ minimalizujemy w optymalizacji

    # Początkowy punkt startowy (środek przedziałów)
    initial_guess = [(b[0] + b[1]) / 2 for b in bounds]

    # Ograniczenia w formacie scipy.optimize
    constraints = [(b[0], b[1]) for b in bounds]

    # Optymalizacja
    result = minimize(objective_function, initial_guess, bounds=constraints, method='SLSQP')

    if result.success:
        best_solution = result.x
        best_utility = -result.fun  # Odpowiada maksymalnej użyteczności
        best_utility_vals = UTA_function_continuous([best_solution], reference_points, lambda_param=lambda_param, use_polynomial=use_polynomial)
        return best_solution, best_utility, best_utility_vals
    else:
        raise ValueError("Optymalizacja nie powiodła się.")


# Przykład użycia (wersja ciągła)
values_discrete = np.array([
    [10, 20, 30, 60],
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_continuous = np.array([
    [80, 90, 100, 10],  # Najgorszy
    [50, 60, 70, 80],   # Neutralny
    [20, 30, 40, 55],  # Najlepszy
])

# values_discrete = np.array([[20, 30], [50, 60], [80, 90], [35, 45], [65, 70]])  # Przykładowe wartości alternatyw
# reference_points_continuous = np.array([[20, 30], [80, 90], [50, 60]])  # Punkty referencyjne (najlepszy, najgorszy, neutralny)
lambda_param = 1  # Przykładowy współczynnik λ (preferencje decydenta)

best_solution_continuous, best_utility_continuous_sum, best_utility_continuous_vals = UTAstar_continuous(values_discrete, reference_points_continuous, lambda_param=lambda_param, use_polynomial=True)

print(f"Najlepsza alternatywa (ciągła): {best_solution_continuous} z użytecznością {best_utility_continuous_sum} | {best_utility_continuous_vals}")


Najlepsza alternatywa (ciągła): [10.         20.         30.         50.92106247] z użytecznością 5.020071010860365 | [[1.33333333 1.33333333 1.33333333 1.02007101]]
